In [1]:
include("../src/KagomeTransport.jl");
using .KagomeTransport.Kagome_Hamiltonian;
using LinearAlgebra;
using Symbolics;
using Symbolics:build_function

In [2]:
# Start with the simple nearest neighbor model with no spin-orbit coupling. Compute the velocity matrices 
@inline function ϕ1(k1, k2)
    1.0 + exp(im * k1)
end

@inline function ϕ2(k1, k2)
    1.0 + exp(im * k2)
end

@inline function ϕ3(k1, k2)
    1.0 + exp(im * (k2 - k1))
end

ϕ3 (generic function with 1 method)

In [3]:
# Here, we use symbolic expressions to derive the orbital velocities 
@variables k1 real = true
@variables k2 real = true 
@variables kx real = true
@variables ky real = true ;


In [4]:
kred = real_basis * [kx, ky]
k1 = kred[1]; k2 = kred[2]
print("k1", " = ", k1, "\n")
print("k2", " = ", k2, "\n")

k1 = kx
k2 = 0.5kx + 0.8660254037844386ky


In [11]:
Φ1nn = 1.0 + exp(im * k1)
Φ2nn = 1.0 + exp(im * k2)
Φ3nn = 1.0 + exp(-im * (k1 - k2));

Φ1nnn = exp(im * (k1 - k2)) + exp(im * k2);
Φ2nnn = exp(-im * (k1 - k2)) + exp(im * k1);
Φ3nnn = exp(-im * k1) + exp(im * k2);

In [12]:
println("Φ1nn = ", Φ1nn, "\n")
println("Φ2nn = ", Φ2nn, "\n")
println("Φ3nn = ", Φ3nn, "\n")

print("\n")

println("Φ1nnn = ", Φ1nnn, "\n")
println("Φ2nnn = ", Φ2nnn, "\n")
println("Φ3nnn = ", Φ3nnn, "\n")



Φ1nn = 1.0 + cos(kx) + im*sin(kx)

Φ2nn = 1.0 + cos(0.5kx + 0.8660254037844386ky) + im*sin(0.5kx + 0.8660254037844386ky)

Φ3nn = 1.0 + cos(-0.5kx + 0.8660254037844386ky) + im*sin(-0.5kx + 0.8660254037844386ky)


Φ1nnn = cos(0.5kx + 0.8660254037844386ky) + cos(0.5kx - 0.8660254037844386ky) + im*(sin(0.5kx + 0.8660254037844386ky) + sin(0.5kx - 0.8660254037844386ky))

Φ2nnn = cos(kx) + cos(-0.5kx + 0.8660254037844386ky) + im*(sin(-0.5kx + 0.8660254037844386ky) + sin(kx))

Φ3nnn = cos(-kx) + cos(0.5kx + 0.8660254037844386ky) + im*(sin(-kx) + sin(0.5kx + 0.8660254037844386ky))



In [7]:
# Symbolic Differentiation 
Dx = Differential(kx); Dy = Differential(ky);

In [8]:
dΦ1dkx = simplify(expand_derivatives(Dx(Φ1)))
dΦ1dky = simplify(expand_derivatives(Dy(Φ1)))

dΦ2dkx = simplify(expand_derivatives(Dx(Φ2)))
dΦ2dky = simplify(expand_derivatives(Dy(Φ2)))

dΦ3dkx = simplify(expand_derivatives(Dx(Φ3)))
dΦ3dky = simplify(expand_derivatives(Dy(Φ3)));


In [9]:
println("dΦ1dkx = ", dΦ1dkx, "\n")
println("dΦ1dky = ", dΦ1dky, "\n")

println("dΦ2dkx = ", dΦ2dkx, "\n")
println("dΦ2dky = ", dΦ2dky, "\n")

println("dΦ3dkx = ", dΦ3dkx, "\n")
println("dΦ3dky = ", dΦ3dky, "\n")

dΦ1dkx = -sin(kx) + im*cos(kx)

dΦ1dky = 0

dΦ2dkx = -0.5sin(0.5kx + 0.8660254037844386ky) + 0.5im*cos(0.5kx + 0.8660254037844386ky)

dΦ2dky = -0.8660254037844386sin(0.5kx + 0.8660254037844386ky) + 0.8660254037844386im*cos(0.5kx + 0.8660254037844386ky)

dΦ3dkx = 0.5sin(-0.5kx + 0.8660254037844386ky) - 0.5im*cos(-0.5kx + 0.8660254037844386ky)

dΦ3dky = -0.8660254037844386sin(-0.5kx + 0.8660254037844386ky) + 0.8660254037844386im*cos(-0.5kx + 0.8660254037844386ky)



In [10]:
# Remark:  k2  = -1/2 kx - 1 / 2 sqrt(3) ky          -0.5 * kx - 0.5 * sqrt(3.0) * ky
#       : -k3  =  1/2 kx - 1 / 2 sqrt(3) ky           0.5 * kx - 0.5 * sqrt(3.0) * ky

@inline function fdΦ1dkx(kx, ky)
    -1.0 * (sin(kx) + im * cos(kx))
end

@inline function fdΦ1dky(kx, ky)
    0.0
end



@inline function fdΦ2dkx(kx, ky)
    0.5 * (sin(-0.5 * kx - 0.5 * sqrt(3.0) * ky) + im * cos(-0.5 * kx - 0.5 * sqrt(3.0) * ky))
end

@inline function fdΦ2dky(kx, ky)
    0.5 * sqrt(3.0) * (sin(-0.5 * kx - 0.5 * sqrt(3.0) * ky) - im * cos(-0.5 * kx - 0.5 * sqrt(3.0) * ky))
end



@inline function fdΦ3dkx(kx, ky)
    0.5 * (-sin(0.5 * kx - 0.5 * sqrt(3.0) * ky) + im * cos(0.5 * kx - 0.5 * sqrt(3.0) * ky))
end

@inline function fdΦ3dky(kx, ky)
    0.5 * sqrt(3.0) * (sin(0.5 * kx - 0.5 * sqrt(3.0) * ky) - im * cos(0.5 * kx - 0.5 * sqrt(3.0) * ky))
end



fdΦ3dky (generic function with 1 method)